# Auto Encoder with MNIST

* Implemented by [`tf.keras.layers`](https://www.tensorflow.org/api_docs/python/tf/keras/layers) and [`eager execution`](https://www.tensorflow.org/guide/eager).

## Import modules

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys
import time
import glob

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import PIL
import imageio
from IPython import display

import tensorflow as tf
from tensorflow.keras import layers
tf.enable_eager_execution()

sys.path.append(os.path.dirname(os.path.abspath('.')))
from utils.image_utils import *
from utils.ops import *

os.environ["CUDA_VISIBLE_DEVICES"]="0"

## Setting hyperparameters

In [ ]:
# Training Flags (hyperparameter configuration)
model_name = 'ae'
train_dir = os.path.join('train', model_name, 'exp1')
max_epochs = 30
save_model_epochs = 5
print_steps = 1000
save_images_epochs = 1
batch_size = 128
learning_rate = 1e-4
num_examples_to_generate = 16
x_dim = 784
latent_dim = 200

## Load the MNIST dataset

In [ ]:
# Load training and eval data from tf.keras
(train_data, train_labels), (test_data, test_labels) = \
    tf.keras.datasets.mnist.load_data()

train_data = train_data.reshape(-1, x_dim).astype('float32')
train_data = train_data / 255.

test_data = test_data.reshape(-1, x_dim).astype('float32')
test_data = test_data / 255.

## Set up dataset with `tf.data`

### create input pipeline with `tf.data.Dataset`

In [ ]:
#tf.set_random_seed(219)

# for train
N = len(train_data)
train_dataset = tf.data.Dataset.from_tensor_slices(train_data)
train_dataset = train_dataset.shuffle(buffer_size=N)
train_dataset = train_dataset.batch(batch_size=batch_size)
print(train_dataset)

test_dataset = tf.data.Dataset.from_tensor_slices(test_data)
test_dataset = test_dataset.shuffle(buffer_size=N)
test_dataset = test_dataset.batch(batch_size=num_examples_to_generate)
print(test_dataset)

## Create the AutoEncoder models

In [ ]:
class AutoEncoder(tf.keras.Model):
  """Build a autoencoder model."""
  def __init__(self, latent_dim=latent_dim, x_dim=x_dim):
    super(AutoEncoder, self).__init__()
    self.fc1 = layers.Dense(units=latent_dim, activation='relu')
    self.fc2 = layers.Dense(units=x_dim, activation='sigmoid')

  def call(self, inputs):
    """Run the model."""
    fc1 = self.fc1(inputs)
    fc2 = self.fc2(fc1)
    
    return fc2

In [ ]:
ae = AutoEncoder()

In [ ]:
# tf.contrib.eager.defun will be deprecated in TF version 2.0
# Defun for performance boost
ae.call = tf.contrib.eager.defun(ae.call)

## Define the loss functions and the optimizer

In [ ]:
def mse_loss(inputs, targets):
  mse_loss = tf.reduce_mean(tf.reduce_sum((inputs - targets)**2, axis=1))
  return mse_loss

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate)

## Checkpoints (Object-based saving)

In [ ]:
checkpoint_dir = train_dir
if not tf.gfile.Exists(checkpoint_dir):
  tf.gfile.MakeDirs(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer, ae=ae)

## Training

In [ ]:
# keeping the random image constant for generation (prediction) so
# it will be easier to see the improvement of the autoencoder.
for inputs in test_dataset.take(1):
  constant_test_input = inputs

In [ ]:
print('Start Training.')
global_step = tf.train.get_or_create_global_step()
num_batches_per_epoch = int(N / batch_size)
for epoch in range(max_epochs):
  
  for step, images in enumerate(train_dataset):
    start_time = time.time()
    
    with tf.GradientTape() as tape:
      reconstruction = ae(images)
      loss = mse_loss(images, reconstruction)
      
    gradients = tape.gradient(loss, ae.variables)
    optimizer.apply_gradients(zip(gradients, ae.variables), global_step=global_step)
    
    epochs = epoch + step / float(num_batches_per_epoch)
    duration = time.time() - start_time

    if global_step.numpy() % print_steps == 0:
      display.clear_output(wait=True)
      examples_per_sec = batch_size / float(duration)
      print("Epochs: {:.2f} global_step: {} loss: {:.3f} ({:.2f} examples/sec; {:.3f} sec/batch)".format(
                epochs, global_step.numpy(), loss.numpy(), examples_per_sec, duration))
      for images in test_dataset.take(1):
        sample_images = ae(images)
      print_or_save_sample_images_two(images.numpy(),
                                      sample_images.numpy(),
                                      num_examples_to_generate)

  if (epoch + 1) % save_images_epochs == 0:
    display.clear_output(wait=True)
    print("This images are saved at {} epoch".format(epoch+1))
    sample_images = ae(constant_test_input)
    print_or_save_sample_images_two(constant_test_input.numpy(),
                                    sample_images.numpy(),
                                    num_examples_to_generate,
                                    is_save=True, epoch=epoch+1,
                                    checkpoint_dir=checkpoint_dir)

  # saving (checkpoint) the model every save_epochs
  if (epoch + 1) % save_model_epochs == 0:
    checkpoint.save(file_prefix=checkpoint_prefix)
    
print('Training Done.')

In [ ]:
# generating after the final epoch
display.clear_output(wait=True)
sample_images = ae(constant_test_input)
print_or_save_sample_images_two(constant_test_input.numpy(),
                                sample_images.numpy(),
                                num_examples_to_generate,
                                is_save=True, epoch=epoch+1,
                                checkpoint_dir=checkpoint_dir)

## Restore the latest checkpoint

In [ ]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

## Display an image using the epoch number

In [ ]:
display_image(max_epochs, checkpoint_dir)

## Generate a GIF of all the saved images.

In [ ]:
filename = model_name + '.gif'
generate_gif(filename, checkpoint_dir)

In [ ]:
display.Image(filename=filename + '.png')